Downloading the required packages

In [1]:
from yahooquery import Ticker
import yfinance as yf
import pandas as pd
import yahoo_fin as yt
from yahoo_fin.stock_info import get_analysts_info
from yahoo_fin.stock_info import get_balance_sheet
from yahoo_fin.stock_info import get_cash_flow
from yahoo_fin.stock_info import get_income_statement
from yahoo_fin.stock_info import get_financials
from yahoo_fin.stock_info import get_quote_table


Input the stock ticker

In [2]:
ticker = input()

aapl


In [3]:

ticker=ticker.upper()
tick=Ticker(ticker)
stock=yf.Ticker(ticker)
df=stock.info
abc=pd.DataFrame([df])
quote_table=pd.DataFrame.from_dict([get_quote_table(ticker)])
balance_s=tick.balance_sheet()
income_s=tick.income_statement()
cash_s=tick.cash_flow()
balance_statement = get_balance_sheet(ticker)
income_statement = get_income_statement(ticker)
cash_flowstatement=get_cash_flow(ticker)
income_statement=income_statement.transpose()
balance_statement=balance_statement.transpose()
cash_flowstatement=cash_flowstatement.transpose()

Financial Ratios Calculation

In [4]:
#liquidityratios
current_ratio=abc['currentRatio']
quick_ratio=((balance_s['CurrentAssets']-balance_s['Inventory'])/balance_s['CurrentLiabilities'])[-1]
cash_ratio=(balance_s['CashAndCashEquivalents'])/balance_s['CurrentLiabilities']
dayssales_outstanding=(balance_s['AccountsReceivable']/income_s['TotalRevenue'])*365
#profitabilityratios
returnonequity=abc['returnOnEquity']
returnonassets=abc['returnOnAssets']
eps=quote_table['EPS (TTM)']
forwardeps=abc['forwardEps']
gross_profit_margin=abc['grossMargins']
net_profit=abc['grossProfits']/(income_s['TotalRevenue'][-1])
#solvencyratios
debt_equity=balance_s['TotalDebt']/balance_s['CommonStockEquity']
debt_asset=balance_s['TotalDebt']/balance_s['TotalAssets']
equityratio=balance_s['StockholdersEquity']/abc['totalDebt'][0]
financial_leverage=balance_s['TotalAssets']/balance_s['StockholdersEquity']




DCF Calculation

In [5]:
#DCF calculation
outstandingshare=abc['sharesOutstanding']
outstandingshare
req_rate=0.074
g_rate=0.02
cashflowgrowthrate=0.03
years=[1,2,3,4]
freecashflow=cash_s['FreeCashFlow']
freecashflow=freecashflow[:-1]
taxrate=income_s['TaxRateForCalcs'][-1]
riskfreerate=0.0397
rm=0.1
req_rate=riskfreerate+abc['beta']*(rm-riskfreerate)
pretax=income_s['InterestExpense'][-1]/balance_s['TotalDebt'][-1]
req_tax=pretax*(1-taxrate)
equity=balance_s['CommonStockEquity']
equity=equity[-1]
debt=balance_s['TotalDebt'][-1]
v=equity+debt
wacc=((equity/debt)*req_rate)+((debt/v)*req_tax)*(1-taxrate)
growth=cash_s['FreeCashFlow']
growth=growth.pct_change()
growth=growth+1
growth.fillna(0)
d=5
for i in range(0,len(growth)):
    l=growth[i]
    if l<d:
        d=l
presentvalue=[]
futurefreecashflow=[]
for i in range(0,4):
    if i==0:
        futurefreecashflow.append(cash_s['FreeCashFlow'][-1]*d)
    else:
        futurefreecashflow.append(futurefreecashflow[i-1]*d)
v0=futurefreecashflow[-1]*(1+g_rate)/(req_rate-g_rate)
for i in range (0,5):
    if i<4:
        presentvalue.append(futurefreecashflow[i]/((1+wacc)**(i+1)))
    else:
        presentvalue.append(v0/((1+wacc)**4))
s=0
for i in range(0,len(presentvalue)):
    s=s+presentvalue[i]
fairvalue=s/balance_s['ShareIssued'][-1]




Piotroski F Score Calculation

In [6]:
#Piotroski F Score Calculation
#profitability aspect
Pio=0
returnonAssetscalc=income_statement['netIncome']/balance_statement['totalAssets']
if returnonAssetscalc[0]>0:
    Pio=Pio+1
if cash_flowstatement['totalCashFromOperatingActivities'][0]>0:
    Pio=Pio=Pio+1
if returnonAssetscalc[0]>returnonAssetscalc[1]:
    Pio=Pio+1
accrual=cash_flowstatement['totalCashFromOperatingActivities']/balance_statement['totalAssets']
if accrual[0]>returnonAssetscalc[0]:
    Pio=Pio+1
#leverage aspect
leverage=balance_statement['longTermDebt']/balance_statement['totalAssets']
if leverage[0]<leverage[1]:
    Pio=Pio+1
currentratiocalc=balance_statement['totalCurrentAssets']/balance_statement['totalCurrentLiabilities']
if currentratiocalc[0]>currentratiocalc[1]:
    Pio=Pio+1
if balance_s['ShareIssued'][-1]<balance_s['ShareIssued'][-2]:
    Pio=Pio+1
#operating efficiency
grossmargin=income_s['GrossProfit']/income_s['TotalRevenue']
if grossmargin[-2]>grossmargin[-3]:
    Pio=Pio+1
assetturn=income_statement['totalRevenue']/balance_statement['totalAssets']
if assetturn[0]>assetturn[1]:
    Pio=Pio+1

Output

In [7]:
print('\033[1m Liquidity Ratios \033[0m')
#Current Ratio
if current_ratio[0]<1:
    s='Poor'
if current_ratio[0]>1 and current_ratio[0]<1.5:
    s='Satisfactory'
if current_ratio[0]>=1.5:
    s='Good'
print('Current Ratio = %f - %s' %(current_ratio[0],s))
#Quick Ratio
if quick_ratio<1:
    s='Poor'
if quick_ratio>=1:
    s='Good'
print('Quick Ratio = %f - %s' %(quick_ratio,s))
#Cash Ratio
if cash_ratio[-1]<0.5:
    s='Poor'
if cash_ratio[-1]>=1:
    s='Good'
if cash_ratio[-1]>0.5 and cash_ratio[-1]<1:
    s='Satisfactory'
print('Cash Ratio = %f - %s' %(cash_ratio[-1],s))
#dayssalesoutstanding
if dayssales_outstanding[-1]>45:
    s='Poor'
if dayssales_outstanding[-1]<=45:
    s='Good'
print('Days Sales Outstanding (DSO) = %f - %s' %(dayssales_outstanding[-1],s))

print('\033[1m Profitability Ratios \033[0m')
#ROE
if returnonequity[0]<.10:
    s='Poor'
if returnonequity[0]>0.1 and returnonequity[0]<0.15:
    s='Satisfactory'
if returnonequity[0]>=0.15:
    s='Good'
print('Return on Equity (in percentage) = %f - %s' %(returnonequity,s))
#ROA
if returnonassets[0]<.05:
    s='Poor'
if returnonassets[0]>0.05 and returnonassets[0]<0.20:
    s='Good'
if returnonassets[0]>=0.15:
    s='Excellent'
print('Return on Equity (in percentage) = %f - %s' %(returnonassets,s))
#EPS
print('Earnings Per Share= %f' %(eps))
#forwardEPS
print('Forward= %f' %(forwardeps))
#grossprofitmargin
if gross_profit_margin[0]<0.2:
    s='Poor'
if gross_profit_margin[0]>=0.2 and gross_profit_margin[0]<0.5:
    s='Good'
if gross_profit_margin[0]>0.5:
    s='Excellent'
gross_profit_margin1=100*gross_profit_margin[0]
print('Gross Profit Margin (in percentage) = %f - %s' %(gross_profit_margin1,s))
#netprofitmargin
if net_profit[0]<0.05:
    s='Poor'
if net_profit[0]>=0.05 and net_profit[0]<0.15:
    s='Good'
if net_profit[0]>0.15:
    s='Excellent'
net_profit1=100*net_profit[0]
print('Net Profit Margin (in percentage) = %f - %s' %(net_profit1,s))
print('\033[1m Solvency Ratios \033[0m')
#Debt-Equity
if debt_equity[-1]<2.1:
    s='Good'
if debt_equity[-1]>=2.1:
    s='Poor'
print('Debt-Equity = %f - %s' %(debt_equity[-1],s))
#debt-asset
if debt_asset[-1]<2.1:
    s='Good'
if debt_asset[-1]>=2.1:
    s='Poor'
print('Debt-Asset= %f - %s' %(debt_asset[-1],s))
#financial leverage
if financial_leverage[-1]<=1:
    s='Good'
if financial_leverage[-1]>1 and financial_leverage[-1]<=2:
    s='Satisfactory'
if financial_leverage[-1]>2:
    s='Poor'
print('Financial Leverage = %f - %s' %(financial_leverage[-1],s))

print('\033[1m Piotroski Score of the stock is %d \033[0m' %(Pio))

print('\033[1m Fair Share Value Calculated using DCF is %f \033[0m'%(fairvalue))


 Liquidity Ratios 
Current Ratio = 0.865000 - Poor
Quick Ratio = 1.022115 - Good
Cash Ratio = 0.278449 - Poor
Days Sales Outstanding (DSO) = 24.749498 - Good
 Profitability Ratios 
Return on Equity (in percentage) = 1.628160 - Good
Return on Equity (in percentage) = 0.222040 - Excellent
Earnings Per Share= 6.050000
Forward= 6.470000
Gross Profit Margin (in percentage) = 43.313998 - Good
Net Profit Margin (in percentage) = 39.437274 - Excellent
 Solvency Ratios 
Debt-Equity = 1.976843 - Good
Debt-Asset= 0.355323 - Good
Financial Leverage = 5.563512 - Poor
 Piotroski Score of the stock is 7 
 Fair Share Value Calculated using DCF is 128.719383 
